# Connecting Python and ANSYS by PARANSYS #2

In this example the APDL script `BeckCantileverAPDL.inp` is used on ANSYS by PARANSYS. The input variables from the script are `q,l,b,h` and the ouput is the stress on a cantilever beam (`stress`). The difference betwen this example and the past is that now we will create a Python function that will return the stress for some properties.

## Importing all that will be used:

In [1]:
import paransys
import numpy as np

## Setup:
First we need to find the ANSYS executable location (`ansloc`) and define the working directory that ANSYS will use (`workingdir`). I'm using ANSYS 2019 R2 and the working dir is at `C:\Temp\wk`.

In [2]:

ansloc = 'C:\\Program Files\\ANSYS Inc\\v194\\ansys\\bin\\winx64\\ansys194.exe'
workingdir = 'C:\\Temp\\wk'
# Create ans object that is ANSYS
ans = paransys.ANSYS(exec_loc=ansloc, run_location=workingdir, jobname='file', nproc=4, override=True, cleardir=False, add_flags='')
# Activate the output to Python
ans.Info(True)

ANSYS properties defined as:
   Executable file: "C:\Program Files\ANSYS Inc\v194\ansys\bin\winx64\ansys194.exe".
   Working directory: "C:\Temp\wk".
   Jobname: "file".
   Number of processors used: "4".
   Override lock file: "True".
   Clear working directory: "False".
   Additional flags: "".
Now the commands will send a return to Python (like this).


### Set APDL model that will be used:
Here it's at the current directory and have no extra files, so we can do:


In [3]:
ans.SetModel(inputname='BeckCantileverAPDL.inp', extrafiles=[], directory='.')

Input script file and extra files copied to working directory.
   Main APDL script: "BeckCantileverAPDL.inp".
   Extra model files: "[]".
   Input directory: ".".


### Setting parameters that will be setted by Python and the parameters that ANSYS should return:
In this case the input parameters are `q,l,b,h` and the ouput `stress`.


In [4]:
# Set input parameters
ans.CreateVarIn('q')
ans.CreateVarIn('l')
ans.CreateVarIn('b')
ans.CreateVarIn('h')

# Set output parameters
ans.CreateVarOut('stress')

ANSYS input variable "Q" created.
ANSYS input variable "L" created.
ANSYS input variable "B" created.
ANSYS input variable "H" created.
Variable "STRESS" declared as ANSYS output variable.


## Creating the stress function
The function `stress(q, l, b, h)` will use ANSYS to return the stress at the beam. 

In this case the lenght will be 1 and we still need to put values in a NumPy array to pass it.

In [17]:
def stress(q ,l, b, h):
    # To set lenght we have to clear old values
    ans.ClearValues()
    # Analysis lenght
    ans.SetLength(1)

    # Creating the arrays
    q = np.array([q])
    l = np.array([l])
    b = np.array([b])
    h = np.array([h])

    # Passing it to ANSYS
    ans.SetVarInValues('q', q)
    ans.SetVarInValues('l', l)
    ans.SetVarInValues('b', b)
    ans.SetVarInValues('h', h)

    # Running ANSYS
    ans.Run()

    # Accessing the results
    results = ans.GetVarOutValues()
    
    # We have just one analysis so the stress is the first line of the array
    stress = results['STRESS'][0]
    return stress

## Using our function
Let's use our function to evaluate the first analys from past example:

In [29]:
first_stress = stress(q=1.15, l=60.0, b=4.0, h=1.0)
print(f'\n\nFirst Stress = {first_stress}.')

The values of all variables were cleared. Now you can change the length parameter.
Analysis lenght set to 1.
Values of "Q" were set.
Values of "L" were set.
Values of "B" were set.
Values of "H" were set.
Deleting old error log file.
Writing the pdsrun.inp file.
Saving the pdsrun.inp file.
Writing the current.samp file.
Running ANSYS.
Solution is done. It took 0.062761 minutes.
Importing results from PDS results file ("C:\Temp\wk\file_current.pdrs").


First Stress = 3103.706543.
